# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

#unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


Cached file found: C:\Users\User\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\User\AppData\Local\Temp\tmp2pwmw3lk\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\User\Downloads


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [15]:
# YOUR CODE HERE
def data_cleaning(df_train, df_test):
    #Drop "," and "."
    df_train["story"] = df_train["story"].str.replace(".","")
    df_train["story"] = df_train["story"].str.replace(",","")
    df_test["story"] = df_test["story"].str.replace(".","")
    df_test["story"] = df_test["story"].str.replace(",","")
    return df_train

In [16]:
def vectorizer(X_train, vec, n_grm, min_d, max_feat):
    #Choosing vectorizer
    if (vec == "Countvectorizer"):
         vectorizer = CountVectorizer(ngram_range = n_grm, min_df = min_d, max_features = max_feat)
    if (vec == "TfidfVectorizer"):
        vectorizer = TfidfVectorizer(ngram_range = n_grm, min_df = min_d, max_features = max_feat)
    
    return vectorizer

In [17]:
def model_selection(vec_X_train,y_train,mod):
    if (mod == 'KNeighborsClassifier'):
        clf = KNeighborsClassifier()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan','Chebyshev'] }
        
        # initialize a GridSearchCV object with the KNeighborsClassifier model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)
        
        best_estimator = grid_search.best_estimator_

        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)
        
    if (mod == 'MultinomialNB'):
        clf = MultinomialNB()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {
        'alpha': [ 0.1, 1, 10, 100],
        'fit_prior': [False,True]}
        
        # initialize a GridSearchCV object with the MultinomialNB model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)
        
        best_estimator = grid_search.best_estimator_
        
        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)

    if (mod == 'SGDClassifier'):
        clf = SGDClassifier()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'loss': ['hinge', 'log', 'modified_huber'],
        'penalty': ['l2', 'l1', 'elasticnet']}
        
        # initialize a GridSearchCV object with the SGDClassifier model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)
        
        best_estimator = grid_search.best_estimator_

        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)

    if (mod == 'Perceptron'):
        clf = Perceptron()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
        'max_iter': [100, 1000, 10000],
        'eta0': [0.0001, 0.001, 0.01, 0.1, 1] }
        
        # initialize a GridSearchCV object with the Perceptron model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)

        best_estimator = grid_search.best_estimator_
        
        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)

    if (mod == 'LinearSVC'):
        clf = LinearSVC()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'class_weight': [None, 'balanced'] }
        
        # initialize a GridSearchCV object with the LinearSVC model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)
        
        best_estimator = grid_search.best_estimator_
        
        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)
        
    if (mod == 'DecisionTree'):
        clf = DecisionTreeClassifier()
        clf.fit(vec_X_train, y_train)
        
        param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [None, 5, 10],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}

        
        # initialize a GridSearchCV object with the DecisionTree model and the parameter grid
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
        grid_search.fit(vec_X_train,y_train)
        
        best_estimator = grid_search.best_estimator_
        
        f1_scorer = make_scorer(f1_score, average='macro')

        # use cross_val_score with the GridSearchCV object as the estimator
        f1_scores = cross_val_score(grid_search.best_estimator_, vec_X_train, y_train, cv=10, scoring=f1_scorer)

        # compute the average F1 score
        f1_average_score = np.mean(f1_scores)

    return clf, f1_average_score, best_estimator

In [13]:
Vectorim = ["Countvectorizer", "TfidfVectorizer"]
Models = ['KNeighborsClassifier','MultinomialNB','SGDClassifier','Perceptron','LinearSVC','DecisionTree']
best_avg = 0;

data_cleaning(df_train, df_test)

X_train = df_train["story"]
y_train = df_train["gender"]

for vec in Vectorim:
    print ("--------- for "+ vec +"--------")
    #Trying diffrent ngrams
    for n_grm in [(1,2),(1,3), (2,3),(1,4), (2,4)]:
        print("--------- for ngram_range: " , n_grm)
        #Trying diffrent min_df
        for min_d in [3, 5, 7]:
            print("------------ for min_df : ", min_d)
            #Trying diffrent max_features
            for max_feat in [10000, 15000, 20000]:
                print("--------------- for max feature: ", max_feat)
                vect= vectorizer(X_train, vec, n_grm, min_d, max_feat)
                vec_X_train = vect.fit_transform(X_train)
                for mod in Models:
                    clf, average_f1,estimator = model_selection(vec_X_train, y_train, mod)
                    if (average_f1 > best_avg):
                        best_avg = average_f1
                        best_clf = clf
                        print("best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!")
                        best_vec = vect
                        best_estimator = estimator
                    print(mod + " Average f1 : " , average_f1 )
                    print("")
                print("")
best_vec
best_estimator
best_avg

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
...,...,...
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,m
749,שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...,m
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,m
751,החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים ז...,f


--------- for Countvectorizer--------
--------- for ngram_range:  (1, 2)
------------ for min_df :  3
--------------- for max feature:  10000
best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
KNeighborsClassifier Average f1 :  0.5053363132198544

best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
MultinomialNB Average f1 :  0.6261061161860071

best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
SGDClassifier Average f1 :  0.6580178875943171

best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
Perceptron Average f1 :  0.6656605684367933

best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
LinearSVC Average f1 :  0.6730049225781275

DecisionTree Average f1 :  0.5377735426505109


--------------- for max feature:  15000
KNeighborsClassifier Average f1 :  0.5024797424891074

best_vec changing!!!!!!!!!!!!!!!!!!!!!!!!!!
MultinomialNB Average f1 :  0.7180640553552147

SGDClassifier Average f1 :  0.6535944190579968

Perceptron Average f1 :  0.6763570685303323

LinearSVC Average f1 :  0.6842115265554853

DecisionTree Average f1 : 

KNeighborsClassifier Average f1 :  0.47130607876612685

MultinomialNB Average f1 :  0.6583008901492773

SGDClassifier Average f1 :  0.6193701171037986

Perceptron Average f1 :  0.6364303790611238

LinearSVC Average f1 :  0.6204317512966491

DecisionTree Average f1 :  0.6026261275324895


------------ for min_df :  7
--------------- for max feature:  10000
KNeighborsClassifier Average f1 :  0.46598140344145156

MultinomialNB Average f1 :  0.6403783061140583

SGDClassifier Average f1 :  0.5028810883772715

Perceptron Average f1 :  0.6459970060822333

LinearSVC Average f1 :  0.5443571095420274

DecisionTree Average f1 :  0.6302261959621316


--------------- for max feature:  15000
KNeighborsClassifier Average f1 :  0.46598140344145156

MultinomialNB Average f1 :  0.6403783061140583

SGDClassifier Average f1 :  0.5918290541665494

Perceptron Average f1 :  0.6459970060822333

LinearSVC Average f1 :  0.5443571095420274

DecisionTree Average f1 :  0.6212957035618001


--------------- for max 

KNeighborsClassifier Average f1 :  0.4707956631070571

MultinomialNB Average f1 :  0.6913416189962667

SGDClassifier Average f1 :  0.654867267236741

Perceptron Average f1 :  0.6745630671699777

LinearSVC Average f1 :  0.6490125296053441

DecisionTree Average f1 :  0.6026211537829493


------------ for min_df :  5
--------------- for max feature:  10000
KNeighborsClassifier Average f1 :  0.46598140344145156

MultinomialNB Average f1 :  0.7052512230217661

SGDClassifier Average f1 :  0.6687290149085824

Perceptron Average f1 :  0.6663792903179921

LinearSVC Average f1 :  0.6770000589981586

DecisionTree Average f1 :  0.5409803140713887


--------------- for max feature:  15000
KNeighborsClassifier Average f1 :  0.46598140344145156

MultinomialNB Average f1 :  0.7052512230217661

SGDClassifier Average f1 :  0.6553090879836163

Perceptron Average f1 :  0.6663792903179921

LinearSVC Average f1 :  0.6770000589981586

DecisionTree Average f1 :  0.5445179356839465


--------------- for max fe

MultinomialNB Average f1 :  0.5668673118071946

SGDClassifier Average f1 :  0.5878376063572838

Perceptron Average f1 :  0.628626711618093

LinearSVC Average f1 :  0.5904792531644035

DecisionTree Average f1 :  0.5722560325709124


--------- for ngram_range:  (1, 4)
------------ for min_df :  3
--------------- for max feature:  10000
KNeighborsClassifier Average f1 :  0.6173863509249549

MultinomialNB Average f1 :  0.7107280325783376

SGDClassifier Average f1 :  0.6746133516133356

Perceptron Average f1 :  0.6759137490930474

LinearSVC Average f1 :  0.6811768142451882

DecisionTree Average f1 :  0.5376827856677094


--------------- for max feature:  15000
KNeighborsClassifier Average f1 :  0.6181427082938852

MultinomialNB Average f1 :  0.711879942627727

SGDClassifier Average f1 :  0.6478450535797743

Perceptron Average f1 :  0.681918573380864

LinearSVC Average f1 :  0.6453725978489335

DecisionTree Average f1 :  0.5491303332730006


--------------- for max feature:  20000
KNeighbors

CountVectorizer(max_features=15000, min_df=3, ngram_range=(1, 3))

MultinomialNB(alpha=1, fit_prior=False)

0.7324259109206436

In [19]:
X_new_vect = best_vec.transform(df_test['story'])

# Use the trained model to make predictions on the vectorized text of the new stories
y_new_pred = best_estimator.predict(X_new_vect)

In [20]:
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_new_pred})
df_predicted.head(5)
df_predicted.tail(5)

unique_values, value_counts = np.unique(y_new_pred, return_counts=True)

#print the unique values and counts
for i in range(len(unique_values)):
    print(f"{unique_values[i]}: {value_counts[i]}")

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


f: 40
m: 283


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [16]:
# df_predicted.to_csv('classification_results.csv',index=False)